In [160]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import os

In [161]:
folder_path = "/Users/andreabelvisi/Documents/GitHub/ML-project-2/Data/monthly_data"
files = os.listdir(folder_path)

In [162]:
datasets = []

for file in files:
    if file == '.DS_Store':
        continue
    df = pd.read_csv(folder_path + '/' + file,  encoding='utf-8')
    df['name'] = file
    datasets.append(df)

datasets.sort(key=lambda x: x['name'][0])

In [163]:
macro = pd.read_csv('Data/macro_data_amit_goyal.csv', encoding='utf-8')
macro = macro[macro['yyyymm']>201500]

In [164]:
def standardize(df):
    return (df - df.mean()) / df.std()

In [165]:
def fill_missing(df):
    return df.fillna(0)

In [166]:
data = []
ret = []

N = 100

for i,df in enumerate(datasets):
    
    df['mcap'] = df['SHROUT'] * df['prc']
    df.drop(['permno', 'DATE', 'Unnamed: 0', 'mve0', 'prc', 'SHROUT', 'sic2', 'name'], axis=1, inplace=True)
    df.dropna(thresh=60, axis=0, inplace=True)
    df = df[df['RET'] > -1]
    df = df.sort_values(by=['mcap'], ascending=False)
    df.drop(['mcap'], axis=1, inplace=True)
    df = df.head(N)
    ret.append(df['RET']-macro['Rfree'].values[i])
    df = df.drop(['RET'], axis=1)
    df = standardize(df)
    df = fill_missing(df)
    data.append(df)

T = len(data)

In [167]:
def solve_f(ret, data, gamma, idx):
    # risolve per la singola f poi dobbiamo metterle in una lista
    return np.linalg.solve(gamma.T@data[idx-1].values.T@data[idx-1].values@gamma, gamma.T@data[idx-1].values.T@ret[idx].values)

In [168]:
def solve_gamma(ret, data, f):
    # f viene passato come lista
    A = np.sum([np.kron(data[i].values.T@data[i].values, f[i].reshape(-1,1)@f[i].reshape(1,-1)) for i in range(len(data)-1)], axis=0)
    B = np.sum([np.kron(data[i].values,f[i].reshape((1,-1))).T@ret[i+1] for i in range(len(data)-1)], axis=0)
    vec_gamma = np.linalg.solve(A, B)
    return vec_gamma.reshape((94, len(f[0])))

Parameter initialization

In [169]:
x = []
for t in range(len(ret)-1):
    x.append(data[t].values.T@ret[t+1].values/len(ret[0]))

x_cov = np.sum([x[i].reshape((-1,1))@x[i].reshape((1,-1)) for i in range(len(x))], axis = 0)
eigval_x, eigvec_x = np.linalg.eig(x_cov)

idx = np.argsort(np.abs(eigval_x))[::-1]
sort_eigvec_x = eigvec_x[:,idx].real
k = 5
gamma = sort_eigvec_x[:,:k]
gamma_reg = sort_eigvec_x[:,:k]
gamma_reg_w = sort_eigvec_x[:,:k]

Regression no weights no regularization

In [171]:
first = False 
while True:

    temp = []
    f_list_new = []

    for i in range(len(data)-1):
        f = solve_f(ret, data, gamma, i+1)
        f_list_new.append(f)
        if first:
            f_change = f-f_list[i]
            temp.append(np.max(f_change))
    first = True
    f_list = f_list_new.copy()

    gamma_new = solve_gamma(ret, data, f_list)
    gamma_change = np.abs(gamma_new-gamma)
    temp.append(np.max(gamma_change))
    gamma = gamma_new.copy()
    
    if (max(temp)<=1e-3):
        break

In [ ]:
#for i in range(1000):
#    f_list = []
#    for i in range(len(data)-1):
#        f_list.append(solve_f(ret, data, gamma, i+1))
#    gamma = solve_gamma(ret, data, f_list)
    

regression with regularization, no weights

In [ ]:
def solve_f_reg(ret, data, gamma, idx, lambda_):
    # risolve per la singola f poi dobbiamo metterle in una lista
    return np.linalg.solve(gamma.T@data[idx-1].values.T@data[idx-1].values@gamma + lambda_*np.eye(gamma.shape[1]), gamma.T@data[idx-1].values.T@ret[idx].values)

In [ ]:
def solve_gamma_reg(ret, data, f, lambda_):
    # f viene passato come lista
    A = np.sum([np.kron(data[i].values.T@data[i].values, f[i].reshape(-1,1)@f[i].reshape(1,-1)) for i in range(len(data)-1)], axis=0) + lambda_*np.eye(gamma.shape[0]*gamma.shape[1])
    B = np.sum([np.kron(data[i].values,f[i].reshape((1,-1))).T@ret[i+1] for i in range(len(data)-1)], axis=0)
    vec_gamma = np.linalg.solve(A, B)
    return vec_gamma.reshape((94, len(f[0])))

In [ ]:
lambda1 = 0.1
lambda2 = 0.1
first = False 
while True:

    temp = []
    f_list_new = []

    for i in range(len(data)-1):
        f = solve_f_reg(ret, data, gamma_reg, i+1, lambda1)
        f_list_new.append(f)
        if first:
            f_change = f-f_list_reg[i]
            temp.append(np.max(f_change))
    first = True
    f_list_reg = f_list_new.copy()

    gamma_new = solve_gamma_reg(ret, data, f_list_reg, lambda2)
    gamma_change = np.abs(gamma_new-gamma_reg)
    temp.append(np.max(gamma_change))
    gamma_reg = gamma_new.copy()
    if (max(temp)<=1e-3):
        break

In [ ]:
#for i in range(100):
#    f_list_reg = []
#    for i in range(len(data)-1):
#        f_list_reg.append(solve_f_reg(ret, data, gamma_reg, i+1, lambda1))
#    gamma_reg = solve_gamma_reg(ret, data, f_list_reg, lambda2)

regression with regularization and weights

In [ ]:
W = np.eye(N)

W_list = [W]*(len(data)-1)

In [ ]:
def solve_f_reg_w(ret, data, gamma, idx,lambda_, W ):
    # risolve per la singola f poi dobbiamo metterle in una lista
    return np.linalg.solve(gamma.T@data[idx-1].values.T@W@data[idx-1].values@gamma + lambda_*np.eye(gamma.shape[1]), 
                           gamma.T@data[idx-1].values.T@W@ret[idx].values)

In [ ]:
def solve_gamma_reg_w(ret, data, f,lambda_, W):
    # f viene passato come lista
    A = np.sum([np.kron(data[i].values.T@W[i]@data[i].values, f[i].reshape(-1,1)@f[i].reshape(1,-1)) for i in range(len(data)-1)], 
               axis=0) + lambda_*np.eye(gamma.shape[0]*gamma.shape[1])
    
    B = np.sum([np.kron(np.sqrt(W[i])@data[i].values,f[i].reshape((1,-1))).T@np.sqrt(W[i])@ret[i+1] for i in range(len(data)-1)], axis=0)

    vec_gamma = np.linalg.solve(A, B)
    return vec_gamma.reshape((94, len(f[0])))

In [ ]:
lambda1 = 0.1
lambda2 = 0.1
first = False 
while True:

    temp = []
    f_list_new = []

    for i in range(len(data)-1):
        f = solve_f_reg_w(ret, data, gamma_reg_w, i+1, lambda1, W_list[i])
        f_list_new.append(f)
        if first:
            f_change = f-f_list_reg_w[i]
            temp.append(np.max(f_change))
    first = True
    f_list_reg_w = f_list_new.copy()

    gamma_new = solve_gamma_reg_w(ret, data, f_list_reg_w, lambda2, W_list)
    gamma_change = np.abs(gamma_new-gamma_reg_w)
    temp.append(np.max(gamma_change))
    gamma_reg_w = gamma_new.copy()
    if (max(temp)<=1e-3):
        break

valuation metrics

In [142]:
def total_R_squared(ret, data, gamma, f_list):
    sum = 0
    ret_2 = 0
    l = len(ret[0])
    for t in range(T-1):
        for i in range(l):
            sum += (ret[t+1].iloc[i] - data[t].iloc[i].values@gamma@f_list[t])**2
            ret_2 += ret[t+1].iloc[i]**2
    
    return 1 - sum/ret_2

In [ ]:
def pred_R_squared(ret, data, gamma, f_list):

    lambda_t = np.mean(np.array(f_list), axis = 0)
    sum = 0
    ret_2 = 0
    l = len(ret[0])
    for t in range(T-1):
        for i in range(l):
            sum += (ret[t+1].iloc[i] - data[t].iloc[i].values@gamma@lambda_t)**2
            ret_2 += ret[t+1].iloc[i]**2
    
    return 1 - sum/ret_2

In [ ]:
print(total_R_squared(ret, data, gamma, f_list))

0.1052400899951863


In [ ]:
print(total_R_squared(ret, data, gamma_reg, f_list_reg))

0.10418398722060174


In [ ]:
print(total_R_squared(ret, data, gamma_reg_w, f_list_reg_w))

0.10418398722060251


In [ ]:
print(pred_R_squared(ret, data, gamma, f_list))

-0.0019198165602634099


In [ ]:
print(pred_R_squared(ret, data, gamma_reg, f_list_reg))

-0.0009789326297886536


In [ ]:
print(pred_R_squared(ret, data, gamma_reg_w,f_list_reg_w))

-0.0009789326297888756


## Kernel regressions:

First step: compute g

In [172]:
def kernel(x,xt, tk=0, alpha=1, l=1, gamma=1):

    norm=np.linalg.norm(x-xt)**2

    #Linear
    if tk==0:
        return x@xt.T
    
    #Gaussian
    if tk==1:
        return np.exp(-norm/(2*l**2))
    
    #Rational Quadratic
    if tk==2:
        return (1+norm/(2*alpha*l**2))**(-alpha)
    
    #Gaussian Exponential
    if tk==3:
        return np.exp(-norm**(gamma/2)/(l**gamma))

In [173]:
def Kernel(x,xt, tk=0, l=1):
    #x is a 1x94 or a 100x94
    #xt is always 100x94

    #Linear
    if tk==0:
        return x@xt.T 

    #Gaussian
    if tk==1:
        K=np.zeros([x.shape[0], xt.shape[0]])
        for i in range(0,x.shape[0]):
            for j in range(0,xt.shape[0]):
                    K[i,j]=np.exp(-np.linalg.norm(x[i,:]-xt[j,:])**2/2*l**2)
        return K


In [174]:
def solve_v_kernel (ret, lambda1, data, f_list):
    #f is a list
    #compute Q matrix
    T=len(data)
    A=np.array(f_list)@np.array(f_list).T
    Q=np.zeros([(T-1)*N, (T-1)*N])
    R=np.array(ret[1:]).flatten()
    Omega=np.eye((T-1)*N, (T-1)*N)

    for t in range(0,T-1):
        for s in range(0,T-1):
            Q[t*100:(t+1)*100,s*100:(s+1)*100]=A[t,s]*kernel(np.array(data[t].values), np.array(data[s].values))
    
    v=np.linalg.solve(Q+lambda1*Omega, R)
    return v

In [175]:
def solve_g_kernel(data, f_list, v, args):
    T=len(data)
    g=np.zeros([np.array(args).shape[0], len(f_list[0])])

    for s in range(0,T-1):
        g=g+(Kernel(args, data[s].values)@v[s*100:(s+1)*100]).reshape(-1,1)@f_list[s].reshape(1,-1) #remember that f_list[s] is F_{s+1} 
    
    return g

Second step: compute factors

In [177]:
def Gram_matrix(data,v, f_list, idx):
    G=solve_g_kernel(data, f_list, v, data[idx].values)
    return G@G.T

In [178]:
def solve_f(ret, v, lambda2, data, f_list):
    T=len(data)
    Omega=np.eye(N, N)
    f_list_new=[]

    for t in range(0,T-1):
        c=np.linalg.solve(Gram_matrix(data,v, f_list, t)+lambda2*Omega, ret[t+1])
        f_list_new.append(solve_g_kernel(data, f_list, v, data[t].values).T@c)
    f_list=f_list_new.copy()
    return f_list
        

Regress:

In [179]:
lambda1 = 0.1
lambda2 = 0.1
for i in range(10):
    print(i)
    v = solve_v_kernel(ret, lambda1, data, f_list)
    f_list = solve_f(ret, v, lambda2, data, f_list)

0
1
2
3
4
5
6
7
8
9


## Low Rank Approximation

In [259]:
def pivoted_chol(K, m_hat):

    d = np.diag(K).reshape(-1,1)
    p_max = np.argmax(d)
    d_max = d[p_max]

    e = np.zeros(len(K)).reshape(-1,1)
    e[p_max] = 1

    L = np.sqrt(1/d_max) * K@e
    B = np.sqrt(1/d_max) * np.eye(len(K))@e

    d = d-L*L

    for m in range(1,m_hat):
        print(m)
        p_max = np.argmax(d)
        d_max = d[p_max]

        e = np.zeros(len(K)).reshape(-1,1)
        e[p_max] = 1

        l = np.sqrt(1/d_max) * (K-L@L.T)@e
        b = np.sqrt(1/d_max) * (np.eye(len(K))-B@L.T)@e

        L = np.concatenate((L,l), axis = 1)
        B = np.concatenate((B, b), axis = 1)

        d = d-l*l

    return L,B

In [232]:
data2=data.copy()

In [246]:
data2=np.array(np.array(data2).reshape(72*100,94)) #flatten data, build X
K=kernel(data2,data2)

In [309]:
K.shape

(7200, 7200)

In [335]:
m_hat=27
L,B=pivoted_chol(K,m_hat)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26


In [336]:
Kernel(data[1].values, data2).shape

(100, 7200)

In [337]:
B.T.shape

(27, 7200)

In [338]:
Phi=B.T@Kernel(data[1].values, data2).T

In [339]:
Phi.shape

(27, 100)

In [325]:

v=np.zeros([m_hat*len(f_list[0]), 1])
lambda1=0.1
Omega=np.eye(N)

#Compute

V = np.sum([np.kron(B.T@Kernel(data[i].values, data2).T@Omega@(B.T@Kernel(data[i].values, data2).T).T, f_list[i].reshape(-1,1)@f_list[i].reshape(1,-1)) for i in range(len(data)-1)], 
            axis=0) + lambda1*np.eye(m_hat*len(f_list[0]))

W = np.sum([np.kron(B.T@Kernel(data[i].values, data2).T@Omega,f_list[i].reshape((-1,1)))@ret[i+1] for i in range(len(data)-1)], axis=0)

v =np.linalg.solve(V, W)



In [340]:
v.reshape(-1,5).shape

(27, 5)

In [344]:
#Compute g(x_1):
g=(B.T@Kernel(data[1].values, data2).T).T@v.reshape(-1,5)

In [346]:
#Compute Gram matrix
G=g@g.T